In [1]:
import os

import dotenv
import pathlib

from src.data import (
    build_stations_dataframe,
    get_k_closest_stations,
    load_json_data_in_date_range,
    build_dock_bikes_timeseries_dataframe,
)

dotenv.load_dotenv()

DATA_RAW_PATH = pathlib.Path(os.getenv("RAW_DATA_PATH"))
DATA_PROCESSED_PATH = pathlib.Path(os.getenv("PROCESSED_DATA_PATH"))
NUM_NEIGHBOR_STATIONS = int(os.getenv("NUM_NEIGHBOR_STATIONS"))
TARGET_STATION_ID = int(os.getenv("TARGET_STATION_ID"))

In [2]:
DATA_RAW_PATH

PosixPath('/home/pablo/VSCodeProjects/bicimad/data/raw')

In [3]:
raw_data = load_json_data_in_date_range(DATA_RAW_PATH)
len(raw_data)

8738

In [4]:
stations = build_stations_dataframe(raw_data)
stations.head()

,name,number,address,latitude,longitude,total_bases
id,,,,,,
1,Puerta del Sol A,1a,Puerta del Sol nº 1,40.4172137,-3.7018341,30
2,Puerta del Sol B,1b,Puerta del Sol nº 1,40.41731271011562,-3.701602938060457,30
3,Miguel Moya,2,Calle Miguel Moya nº 1,40.4205886,-3.7058415,24
4,Plaza Conde Suchil,3,Plaza del Conde del Valle de Súchil nº 3,40.4302937,-3.7069171,18
5,Malasaña,4,Calle Manuela Malasaña nº 5,40.4285524,-3.7025875,24


In [5]:
neighbor_selected_stations = get_k_closest_stations(
    target_station_id=TARGET_STATION_ID,
    stations=stations,
    k=NUM_NEIGHBOR_STATIONS,
)
neighbor_selected_stations.head()

,name,number,address,latitude,longitude,total_bases,distance
id,,,,,,,
102,Alcalá,97,Avenida de Menéndez Pelayo nº 3,40.4222969,-3.6805189,27,0.098442
101,Castelló,96,Calle Alcalá nº 111,40.4220640,-3.6821793,27,0.194586
107,Velázquez,102,Calle Alcalá nº 95,40.4211802,-3.6840229,24,0.342588
65,Narváez,61,Calle O'Donnell nº 28,40.4213983,-3.6752045,24,0.407816
103,Plaza de Felipe II,98,Plaza de Felipe II,40.4239447,-3.6758383,24,0.445928


In [6]:
selected_stations_ids = [TARGET_STATION_ID]
selected_stations_ids.extend(neighbor_selected_stations.index.to_list())
selected_stations_ids

[90, 102, 101, 107, 65, 103]

In [7]:
dock_bikes_timeseries = build_dock_bikes_timeseries_dataframe(
    data=raw_data,
    station_ids=selected_stations_ids,
)

In [8]:
dock_bikes_timeseries

,90,102,101,107,65,103
timestamps,,,,,,
2022-01-01 00:13:20.603583,19,21,21,14,13,15
2022-01-01 01:13:21.911079,19,20,21,14,11,15
2022-01-01 02:13:23.718951,19,20,20,14,11,13
2022-01-01 03:13:23.902654,18,20,20,14,11,12
2022-01-01 04:13:26.826536,18,20,19,14,11,11
...,...,...,...,...,...,...
2022-12-31 19:57:50.302819,8,12,12,5,11,2
2022-12-31 20:57:50.382304,4,11,11,6,9,2
2022-12-31 21:57:52.761035,3,13,13,6,11,1
